In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
        .builder \
        .appName("streaming") \
        .master("local[4]") \
        .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [2]:
from kafka_test.SparkKafkaConsumer import Comsumer

# 启动消费者
Comsumer(spark).start_consumer()

Spark Consuming...


In [12]:
# 测试是否将采集的数据存储到表中
result = spark.sql("SELECT * FROM testedTable1")
print(result.count())
result.show()

46
+--------+--------------------+--------------------+----------+-----------+--------+--------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+
|order_id|                  id|                name|popularity|duration_ms|explicit|             artists|          id_artists|release_date|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|  tempo|time_signature|           timestamp|
+--------+--------------------+--------------------+----------+-----------+--------+--------------------+--------------------+------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-------+--------------+--------------------+
|       0|3BFRqZFLSrqtQr6cj...|   Ain't Misbehavin'|        51|   237773.0|       0|          FatsWaller| DYWCXTkNqGFZIfSrWEa|        1926|       0.515| 0

In [13]:
df = spark.sql("SELECT * FROM testedTable1")
df = df.sort(df.release_date.desc())
# 从采集的数据集中删除不需要的列
df = df.drop('order_id', 'id', 'explicit', 'release_date',
             'id_artists', 'time_signature', 'duration_ms', 'timestamp')
df

DataFrame[name: string, popularity: int, artists: string, danceability: double, energy: double, key: int, loudness: double, mode: int, speechiness: double, acousticness: double, instrumentalness: double, liveness: double, valence: double, tempo: double]

In [14]:
from spotifyRec.song_data_processing import SongDataProcessor
add_df = SongDataProcessor().get_random_song()
df_sp = spark.createDataFrame(add_df)
# 将我们喜欢的音乐加入到这个数据集中  由于两者列名顺序不同，需要使用unionByName(类型按照为df原先的类型)
union_df = df.unionByName(df_sp)

In [16]:
from spotifyRec.song_data_processing import SongDataProcessor

assembled_data = SongDataProcessor().feature_vectorise(union_df)
output_df = SongDataProcessor().kmeans_cluster(assembled_data)
data_df = SongDataProcessor().rec_prior_preparation(output_df)

16


In [17]:
from spotifyRec.recommender import SpotifyRecommender
import pandas as pd

recommend_num = 10
liked_song_name = add_df.iloc[-1]['name'] # 获取用户收藏音乐的名称
value_cate = data_df.iloc[-1]['prediction'] # 用户收藏音乐对应的类别（k-means预测的）
print(liked_song_name, value_cate)

filtered_data = data_df[data_df['prediction'] == 1]
recommender = SpotifyRecommender(filtered_data)
rec_song = recommender.spotify_recommendations(recommend_num)
rec_song

Starlight (Taylor's Version) 2


,name,popularity,artists,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
80,Cheek to Cheek,51,FredAstaire,0.506,0.180,0,-12.184,1,0.0443,0.908,0.000002,0.0652,0.6650,161.510
46,Mona Lisa,56,NatKingCole,0.241,0.237,1,-12.201,1,0.0371,0.952,0.000000,0.1130,0.3030,174.578
78,In the Mood - Live,52,GlennMiller,0.463,0.654,8,-9.720,1,0.0659,0.824,0.917000,0.3720,0.5920,175.204
65,Whatever Will Be Will Be (Que Sera Sera) (with...,62,DorisDayFrankDeVolHisOrchestra,0.416,0.341,9,-10.749,1,0.0391,0.864,0.000000,0.7240,0.7760,177.229
55,Singin' In The Rain,53,GeneKelly,0.354,0.241,5,-13.811,1,0.0334,0.889,0.000274,0.1650,0.4080,135.646
76,White Christmas,56,BingCrosbyKenDarbySingersJohnScottTrotterHisOr...,0.317,0.158,9,-16.485,1,0.0381,0.673,0.353000,0.3380,0.2470,130.503
19,I Saw The Light - Single Version,54,HankWilliams,0.349,0.265,7,-11.381,1,0.0324,0.950,0.000118,0.1650,0.4800,204.656
77,Where Did You Sleep Last Night,53,LeadBelly,0.570,0.241,4,-12.138,1,0.0473,0.948,0.022400,0.1110,0.5430,115.352
20,Manhattan,51,EllaFitzgerald,0.430,0.278,10,-11.064,1,0.0357,0.867,0.000000,0.0892,0.4150,113.579
53,Autumn Leaves - Remastered,52,NatKingCole,0.211,0.348,2,-10.120,0,0.0326,0.939,0.000520,0.3840,0.0981,86.868
